### Steam_games EDA
- 목표 
- game에 장르와 tags에 대해서 다룹니다

In [ ]:
# ====== 1. Environment & Imports ======
import os
import ast
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv

# Settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
load_dotenv()

# Path
DATA_PATH = '../../steam/now_steam/steam_games.json'
print(f"Data Path: {DATA_PATH}")

Data Path: ../../steam/now_steam/steam_games.json


In [2]:
raw_data = open(DATA_PATH,'r',encoding='utf-8')
print(raw_data.readline())

{u'publisher': u'Kotoshiro', u'genres': [u'Action', u'Casual', u'Indie', u'Simulation', u'Strategy'], u'app_name': u'Lost Summoner Kitty', u'title': u'Lost Summoner Kitty', u'url': u'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/', u'release_date': u'2018-01-04', u'tags': [u'Strategy', u'Action', u'Indie', u'Casual', u'Simulation'], u'discount_price': 4.49, u'reviews_url': u'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1', u'specs': [u'Single-player'], u'price': 4.99, u'early_access': False, u'id': u'761140', u'developer': u'Kotoshiro'}



In [9]:
def load_steam_data(filepath):
    rows = []
    error_count = 0
    error_rows = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                # Note: The file format is often python-dict string, not strict JSON
                data = ast.literal_eval(line.strip())
                rows.append(data)
            except (SyntaxError, ValueError):
                error_count +=1
                error_rows.append(i)
                continue
        print(f"{error_count}개의 줄이 파싱되지 못했으며 그 줄은 {error_rows}입니다")
    return pd.DataFrame(rows)
games = load_steam_data(DATA_PATH)

0개의 줄이 파싱되지 못했으며 그 줄은 []입니다


In [17]:
meta_df=games[['genres','tags']]
col_null = meta_df.isnull().sum().to_dict()
print(col_null)

{'genres': 3283, 'tags': 163}


In [21]:
# ====== 3. Cross-Check with User Logs (검증 로직) ======
# Goal: 삭제할 Zombie Game과 남길 Unknown Game 구분하기

import ast

# 1. Identify missing games (태그로도 복구 안 된 것들)
missing_mask = meta_df['genres_filled'].isnull()
missing_ids = set(meta_df[missing_mask]['item_id'])
print(f"Games with Missing Genres (Residual): {len(missing_ids):,}")

# 2. Load Played Game IDs from User Logs
# (메모리 효율을 위해 id만 빠르게 로드)
USER_DATA_PATH = '../../steam/now_steam/australian_users_items.json'
print("Loading User Logs for Verification...")

def get_played_game_ids(filepath):
    played_ids = set()
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = ast.literal_eval(line.strip())
                for item in data['items']:
                    if item['playtime_forever'] > 0: # 0분 플레이는 무시
                        played_ids.add(item['item_id'])
            except:
                continue
    return played_ids

played_games = get_played_game_ids(USER_DATA_PATH)
print(f"Total Unique Games Played by Users: {len(played_games):,}")

# 3. Check Overlap (교집합 확인)
actually_played_missing = missing_ids.intersection(played_games)
zombie_games = missing_ids - played_games

print(f"\n[Verification Result]")
print(f"- Missing but Played (Keep as 'Unknown'): {len(actually_played_missing):,}")
print(f"- Missing & Never Played (Safe to Drop): {len(zombie_games):,}")

# 4. Action: Drop Zombies
# 좀비 게임 제거한 클린 데이터 생성
meta_df_clean = meta_df[~meta_df['id'].isin(zombie_games)].copy()

# 5. Fill remaining missing genres with 'Unknown'
meta_df_clean['genres_filled'] = meta_df_clean['genres_filled'].fillna('Unknown')

print(f"\nFinal Metadata Size: {len(meta_df_clean):,} (Dropped {len(zombie_games)} zombies)")
print(f"Remaining Missing Genres: {meta_df_clean['genres_filled'].isnull().sum()}")

KeyError: 'item_id'

In [ ]:
# [Coach Navi] Tag를 통해서 누락된 장르 복원
# Logic: 장르가 없다면 Tag를 통해서 누락된 장르를 복원한다.

# 1. 모든 장르를 set으로 faltten list로 변환
all_genres = set()
for g_list in meta_df['genres'].dropna():
    all_genres.update(g_list)

print(f"Official Genres List ({len(all_genres)}): {sorted(list(all_genres))[:5]} ...")

# 2. Imputation Function
def impute_genre(row):
   # 장르가 있다면 넣고
    if isinstance(row['genres'], list):
        return row['genres']
    
    # 장르가 있지만 tags가 있을경우
    if isinstance(row['tags'], list):
        # tags-> genres 로 대체
        inferred = [tag for tag in row['tags'] if tag in all_genres]
        if inferred:
            return inferred # 추론된 장르 반환
            
    return row['genres'] # 전처리를 거치고 남은 value

# 3. Apply
print(f"Missing Genres BEFORE: {meta_df['genres'].isnull().sum()}")
meta_df['genres_filled'] = meta_df.apply(impute_genre, axis=1)
print(f"Missing Genres AFTER:  {meta_df['genres_filled'].isnull().sum()}")

# Check improvement
filled_count = meta_df['genres'].isnull().sum() - meta_df['genres_filled'].isnull().sum()
print(f"Successfully Recovered: {filled_count} games")

Official Genres List (22): ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production'] ...
Missing Genres BEFORE: 3283
Missing Genres AFTER:  1031
Successfully Recovered: 2252 games


/tmp/ipykernel_20483/3879978945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_df['genres_filled'] = meta_df.apply(impute_genre, axis=1)
